# Granger causality test tutorial
###### by Daegun Kim

### <span style="color:#ffd33d">**Warning**</span>
> Granger causality test can only provide 'Granger Causality' not the causality   
Thus, it would be stretching a point to use Granger causality test as proof of general causality between two time series variables.

## Load packages

In [ ]:
import numpy as np
import matplotlib as mpl
import pandas as pd
import seaborn as sns

from statsmodels.tsa.stattools import grangercausalitytests
from matplotlib import pyplot as plt

## Configurations

In [ ]:
def fix_random_seed(seed=42):
    import random
    import numpy as np 
    import os

    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    try:
        tf.random.set_seed(seed)
    except:
        pass
    
fix_random_seed()

In [ ]:
from matplotlib import font_manager, rc
try:
    font_path = "C:/Windows/Fonts/malgun.TTF"
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
except:
    pass

# Fix minus presentation
mpl.rcParams['axes.unicode_minus'] = False

# Matplotlib fontsize change
SMALL_SIZE = 10
MEDIUM_SIZE = 15
LARGE_SIZE = 20

plt.rc('font', size=SMALL_SIZE, weight='bold')
plt.rc('axes', titlesize=LARGE_SIZE, titleweight='bold')
plt.rc('axes', labelsize=MEDIUM_SIZE, labelweight='bold')
plt.rc('axes', titleweight='bold')
plt.rc('xtick', labelsize=MEDIUM_SIZE)
plt.rc('ytick', labelsize=MEDIUM_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=LARGE_SIZE)

## Example with causality

In [ ]:
# Set time
t = np.arange(0, 30)

# Set factor
factor_1 = np.sin(t/np.pi)

time_lag = 4
factor_2 = 0.3*t + np.sin((t+time_lag)/np.pi)

# Assign noise to each factor
factor_1_noise = factor_1 + np.random.random(size=np.shape(factor_1))
factor_2_noise = factor_2 + np.random.random(size=np.shape(factor_1))

# Make DataFrame
df_w_cause_w_noise = pd.DataFrame(data={
    'y1':factor_1_noise, 'y2':factor_2_noise
})

In [ ]:
# plot - two time series factors with causality
fig_w_cause, ax_w_cause = plt.subplots(2, 1, figsize=(16, 12))

ax_w_cause[0].plot(t, factor_1, marker='o', label='Factor 1 wo/ noise')
ax_w_cause[0].plot(t, factor_2, marker='o',label='Factor 2 wo/ noise')
ax_w_cause[0].legend()
ax_w_cause[0].set_xlabel('Time')
ax_w_cause[0].grid(True)

ax_w_cause[1].plot(t, factor_1_noise, marker='o',label='Factor 1 w/ noise')
ax_w_cause[1].plot(t, factor_2_noise, marker='o',label='Factor 2 w/ noise')
ax_w_cause[1].legend()
ax_w_cause[1].set_xlabel('Time')
ax_w_cause[1].grid(True)

fig_w_cause.suptitle(f'Example with causality \ny1 -> y2 & lag = {time_lag}', fontsize=20)

In [ ]:
# Test the Granger causality y1->y2
max_lag = 8

gct_w_cause_w_noise_y1_y2 = grangercausalitytests(
    df_w_cause_w_noise[['y2','y1']],
    maxlag=max_lag,
    verbose=2,
)

In [ ]:
# Test the Granger causality y2 -> y1
gct_w_cause_w_noise_y2_y1 = grangercausalitytests(
    df_w_cause_w_noise[['y1','y2']],
    maxlag=max_lag,
    verbose=2,
)

In [ ]:
# Set test resut DataFrame
df_pval_w_cause_w_noise = pd.DataFrame(
    columns=['y1->y2', 'y2->y1'],
    index=np.arange(1, max_lag+1),
    dtype='int',
)

# Take the Ganger causality test result of y1 affected y2
for k1, v1 in gct_w_cause_w_noise_y1_y2.items():
    p_vals_y1_y2 = []

    for k2, v2 in v1[0].items():
        p_vals_y1_y2.append(v2[1])

    if all(np.array(p_vals_y1_y2)<0.05):
        df_pval_w_cause_w_noise.at[k1, 'y1->y2'] = 1
    else:
        df_pval_w_cause_w_noise.at[k1, 'y1->y2'] = -1

# Take the Ganger causality test result of y2 affected y1
for k1, v1 in gct_w_cause_w_noise_y2_y1.items():
    p_vals_y2_y1 = []

    for k2, v2 in v1[0].items():
        p_vals_y2_y1.append(v2[1])

    if all(np.array(p_vals_y2_y1)<0.05):
        df_pval_w_cause_w_noise.at[k1, 'y2->y1'] = 1
    else:
        df_pval_w_cause_w_noise.at[k1, 'y2->y1'] = -1

df_pval_w_cause_w_noise

In [ ]:
# plot - p-value result by time lag
fig_w_cause_heat, ax_w_cause_heat = plt.subplots(1, 1, figsize=(12, 6))

sns.heatmap(
    df_pval_w_cause_w_noise.T, 
    square=True,
    cmap='RdBu',
    alpha=0.8,
    cbar=False,
    ax=ax_w_cause_heat,
    linecolor='white',
    linewidths=2,
    )

ax_w_cause_heat.set_title(f'Blue: significant | Red: Non-significant\nTrue Time lag: {time_lag}  ||  y1->y2', fontsize=20)
ax_w_cause_heat.set_xlabel('Time lag', fontsize=15)
ax_w_cause_heat.set_ylabel('Statistical-significance', fontsize=15)

## Example without causality

In [ ]:
# Set factor
factor_3 = t/15
factor_4 = np.sin(t/np.pi) + np.random.random(size=np.shape(factor_1))

# Assign noise to each factor
factor_3_noise = factor_3 + np.random.random(size=np.shape(factor_1))
factor_4_noise = factor_4 + np.random.random(size=np.shape(factor_1))

# Make DataFrame
df_wo_cause_w_noise = pd.DataFrame(data={
    'y3':factor_3_noise, 'y4':factor_4_noise
})

In [ ]:
# plot - two time series factors with causality
fig_wo_cause, ax_wo_cause = plt.subplots(2, 1, figsize=(16, 12))

ax_wo_cause[0].plot(t, factor_3, marker='o', label='Factor 3 wo/ noise')
ax_wo_cause[0].plot(t, factor_4, marker='o', label='Factor 4 wo/ noise')
ax_wo_cause[0].legend()
ax_wo_cause[0].set_xlabel('Time')
ax_wo_cause[0].grid(True)

ax_wo_cause[1].plot(t, factor_3_noise, marker='o',label='Factor 3 w/ noise')
ax_wo_cause[1].plot(t, factor_4_noise, marker='o',label='Factor 4 w/ noise')
ax_wo_cause[1].legend()
ax_wo_cause[1].set_xlabel('Time')
ax_wo_cause[1].grid(True)

fig_wo_cause.suptitle(f'Example without causality', fontsize=20)

In [ ]:
# Test the Granger causality y3->y4
max_lag = 8

gct_wo_cause_w_noise_y3_y4 = grangercausalitytests(
    df_wo_cause_w_noise[['y3','y4']],
    maxlag=max_lag,
    verbose=2,
)

In [ ]:
# Test the Granger causality y4 -> y3
gct_wo_cause_w_noise_y4_y3 = grangercausalitytests(
    df_wo_cause_w_noise[['y3','y4']],
    maxlag=max_lag,
    verbose=2,
)

In [ ]:
# Set test resut DataFrame
df_pval_wo_cause_w_noise = pd.DataFrame(
    columns=['y3->y4', 'y4->y3'],
    index=np.arange(1, max_lag+1),
    dtype='int',
)

# Take the Ganger causality test result of y1 affected y2
for k1, v1 in gct_wo_cause_w_noise_y3_y4.items():
    p_vals_y3_y4 = []

    for k2, v2 in v1[0].items():
        p_vals_y3_y4.append(v2[1])

    if all(np.array(p_vals_y3_y4)<0.05):
        df_pval_wo_cause_w_noise.at[k1, 'y3->y4'] = 1
    else:
        df_pval_wo_cause_w_noise.at[k1, 'y3->y4'] = -1

# Take the Ganger causality test result of y4 affected y3
for k1, v1 in gct_wo_cause_w_noise_y4_y3.items():
    p_vals_y4_y3 = []

    for k2, v2 in v1[0].items():
        p_vals_y4_y3.append(v2[1])

    if all(np.array(p_vals_y4_y3)<0.05):
        df_pval_wo_cause_w_noise.at[k1, 'y4->y3'] = 1
    else:
        df_pval_wo_cause_w_noise.at[k1, 'y4->y3'] = -1

df_pval_wo_cause_w_noise

In [ ]:
# plot - p-value result by time lag
fig_wo_cause_heat, ax_wo_cause_heat = plt.subplots(1, 1, figsize=(12, 6))

sns.heatmap(
    df_pval_wo_cause_w_noise.T, 
    square=True,
    cmap='RdBu',
    alpha=0.8,
    cbar=False,
    ax=ax_wo_cause_heat,
    linecolor='white',
    linewidths=2,
    )

ax_wo_cause_heat.set_title(f'Blue: significant | Red: Non-significant\nTrue Time lag: [None]', fontsize=20)
ax_wo_cause_heat.set_xlabel('Time lag', fontsize=15)
ax_wo_cause_heat.set_ylabel('Statistical-significance', fontsize=15)

***
### Refence
* https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.grangercausalitytests.html
* https://en.wikipedia.org/wiki/Granger_causality
* https://intothedata.com/02.scholar_category/timeseries_analysis/granger_causality/
* https://www.aptech.com/blog/introduction-to-granger-causality/